In [73]:
import tensorflow as tf
import pydot
from pathlib import Path
import os
import json
import requests
import numpy as np

# Using Keras to load the dataset

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 17s 4us/step


In [3]:
fashion_mnist

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

In [5]:
X_train, y_train =  X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid =  X_train_full[-5000:], y_train_full[-5000:]

In [6]:
X_train.shape

(55000, 28, 28)

In [7]:
X_train.dtype

dtype('uint8')

In [9]:
# we'll scale the pixel intensities down to the 0-1 range
X_train, X_valid, X_test = X_train / 225., X_valid / 225., X_test / 225.

In [11]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [12]:
class_names[y_train[0]]

'Ankle boot'

# Creating the model using the sequential API

In [13]:
tf.random.set_seed(73) # tf.keras.utils.set_random_seed() sets the random seeds TensorFlow, Python and Numpy

# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Input(shape=[28, 28]))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(300, activation="relu"))
# model.add(tf.keras.layers.Dense(100, activation="relu"))
# model.add(tf.keras.layers.Dense(10, activation="softmax"))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

2023-04-21 15:42:23.807868: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-04-21 15:42:23.848286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-21 15:42:23.848495: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.68GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2023-04-21 15:42:23.850366: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-04-21 15:42:23.872705: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-04-21 15:42:23.885832: I tensorflow/stream_executor/plat

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [34]:
# tf.keras.utils.plot_model(model)

Keras ensures the name a layers is globally unique, even across models. This makes it possible to merge models easily without getting name conflicts.

In [36]:
model.layers

In [38]:
hidden1 = model.layers[1]
hidden1.name

'dense'

In [40]:
model.get_layer('dense') is hidden1

True

In [41]:
weights, biases = hidden1.get_weights()
weights

array([[ 0.06204498,  0.00013475,  0.037318  , ...,  0.06567547,
         0.02667287,  0.02535582],
       [-0.06401441, -0.01844299,  0.00234362, ..., -0.00727981,
         0.03055523, -0.03390701],
       [-0.00221441,  0.0231154 , -0.03318587, ...,  0.02216118,
         0.0165878 ,  0.03076063],
       ...,
       [-0.00157608,  0.03542616, -0.02543213, ...,  0.05203415,
        -0.01936731,  0.03244814],
       [ 0.04760689,  0.0428013 ,  0.02297359, ...,  0.06337433,
         0.0532454 ,  0.02867785],
       [-0.00720569, -0.01726631,  0.02810925, ...,  0.02170168,
         0.03227305, -0.0082326 ]], dtype=float32)

In [42]:
weights.shape

(784, 300)

In [43]:
biases

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [44]:
biases.shape

(300,)

In [45]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [46]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30


2023-04-21 18:46:30.920769: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


1719/1719 [==============================] - 2s 1ms/step - loss: 0.6859 - accuracy: 0.7719 - val_loss: 0.5055 - val_accuracy: 0.8242
Epoch 2/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4781 - accuracy: 0.8337 - val_loss: 0.4547 - val_accuracy: 0.8412
Epoch 3/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4337 - accuracy: 0.8475 - val_loss: 0.4009 - val_accuracy: 0.8590
Epoch 4/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4060 - accuracy: 0.8591 - val_loss: 0.3921 - val_accuracy: 0.8626
Epoch 5/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3869 - accuracy: 0.8639 - val_loss: 0.3783 - val_accuracy: 0.8656
Epoch 6/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3693 - accuracy: 0.8702 - val_loss: 0.3660 - val_accuracy: 0.8728
Epoch 7/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3555 - accuracy: 0.8750 - val_loss: 0.3807 - val_accuracy: 0.8660
Epo

In [49]:
model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version

In [51]:
model.save(model_path, save_format="tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


2023-04-21 18:57:52.461687: W tensorflow/python/util/util.cc:329] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [52]:
!saved_model_cli show --dir my_mnist_model/0001

The given SavedModel contains the following tag-sets:
'serve'


In [53]:
!saved_model_cli show --dir my_mnist_model/0001 --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


__saved_model_init_op --> an initialization function

serving_default --> defult serving function

In [54]:
!saved_model_cli show --dir my_mnist_model/0001 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_2'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


# TensorFlow Serving

Run install-tf-server.sh 

In [59]:
%pip install -q -U tensorflow-serving-api

Note: you may need to restart the kernel to use updated packages.


In [65]:
os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [67]:
%%bash --bg
tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=my_mnist_model --model_base_path="${MODEL_DIR}" > my_server.log 2>&1

It is now waiting for gRPC and REST requests, respectively, on ports 8500 and 8501

# Querying TF Serving through the REST API

In [76]:
X_new = X_test[:6] # pretend we have 3 new digit images to classify
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [77]:
server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [78]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.97],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.35, 0.  , 0.  , 0.  , 0.  , 0.  , 0.64, 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [79]:
y_test[:6]

array([9, 2, 1, 1, 6, 1], dtype=uint8)